In [ ]:
t_start = timeit.default_timer()

mean_var = copy.copy(n_list)
for model in model_list: #model
    for ic, c in enumerate(c_list): #command    
        for im, m in enumerate(m_list): #movement
            if model_cm[model,c,m,'pool_match_success']:    
                print(model,c,m)
                c_idxs = model_cm[model,c,m,'pool_match_idx']#use these idxs for average
                mu_c = df_lqr.loc[c_idxs, mean_var].mean()

                #MOVE:
                cm_sel = model_cm[model,c,m,'sel']
                cm_idxs = cm_sel[cm_sel].index.values
                mu_cm = df_lqr.loc[cm_idxs, mean_var].mean()       

                #SHUFFLE:
                nan_mat = np.ones((len(mean_var), num_shuffle))*np.nan
                s_mean = xr.DataArray(nan_mat, 
                                  coords={'v':mean_var,'shuffle':range(num_shuffle)},
                                  dims=['v','shuffle'])
                for s in range(num_shuffle):
                    s_idxs = model_cm[model,c,m,'shuffle_mat'][:,s].astype(int)
                    mu_s = df_lqr.loc[s_idxs, mean_var].mean()
                    s_mean.loc[:,s] = mu_s

                #DIFFERENCE: 

                #ASSIGN:
#                 model_cm[model,c,m,'mat_df'] = df_lqr.loc[cm_idxs, mean_var]
                model_cm[model,c,m,'n_c'] = mu_c
                model_cm[model,c,m,'n_cm'] = mu_cm
  
                diff_i = mu_c-mu_cm   
                diff_potent_i, diff_null_i, _ = bmi_util.proj_null_potent(Kn[2:4,:].T, np.array(diff_i).reshape((-1,1)))
                model_cm[model,c,m,'n_diff_true'] = diff_i             
                model_cm[model,c,m,'n_diff_norm_true'] = np.linalg.norm(diff_i)
                model_cm[model,c,m,'n_diff_norm_potent'] = np.linalg.norm(diff_potent_i)
                model_cm[model,c,m,'n_diff_norm_null'] = np.linalg.norm(diff_null_i)
            
                model_cm[model,c,m,'n_s'] = s_mean              
                n_c_rep = np.array(mu_c)[...,None]
                model_cm[model,c,m,'n_diff_s'] = n_c_rep-s_mean
                model_cm[model,c,m,'n_diff_s_norm'] = np.linalg.norm(model_cm[model,c,m,'n_diff_s'], axis=0)
                model_cm[model,c,m,'n_diff_s_norm_mean'] = model_cm[model,c,m,'n_diff_s_norm'].mean()

t_elapsed = timeit.default_timer()-t_start
print(t_elapsed)

In [ ]:
#Identify movements + num_obs +sel for each command
#also save for each movement separately

bin_buffer = 5 #only count data after the first 5 samples and before the last 5 samples: 
min_obs = 15
cm_dic = {}
#Loop over command:
for model in model_list:
    for c in c_list: 
        bm = c[0]
        ba = c[1]

        cm_dic[model, c] = []
        cm_dic[model, c,'num_obs'] = []
        #Loop over movement: 
        for m in move_list:
            print(model, c, m)
            sel = \
            (df['model']==model) \
            &(df['target']==m[0]) \
            & (df['task_rot']==m[1]) \
            & (df['bin'] >= bin_buffer) \
            & (df['bin_end'] >= bin_buffer) \
            & (df['u_v_mag_bin'] == bm) \
            & (df['u_v_angle_bin'] == ba)
            num_obs = sum(sel)
    #             print(bm, ba, m, num_obs)
            #ASSIGN:
            cm_dic[model,c,m,'sel'] = sel
            cm_dic[model,c,m,'num_obs'] = num_obs
            if num_obs >= min_obs:
                cm_dic[model,c].append(m)
                cm_dic[model,c,'num_obs'].append(num_obs)
#Make the movement-pool sel: 
for model in model_list:
    for c in c_list:
        bm = c[0]
        ba = c[1]
        cm_dic[model, c, 'sel'] = np.zeros(df.shape[0]).astype('bool')
        for m in cm_dic[model,(bm,ba)]:
            cm_dic[model, c, 'sel'] = cm_dic[model, c, 'sel'] \
            | cm_dic[model, c,m,'sel']
    #movement-pool sel wil be useful as the global mean for command-activity.